# ハイパーパラメータチューニングジョブの結果を分析する

チューニングジョブを完了すると（またはジョブが実行中であっても）、このノートブックを使用して結果を分析して、各ハイパーパラメータがモデルの品質にどのように影響するかを理解することができます。

---
## 環境を設定する
分析を開始するには、ハイパーパラメータチューニングジョブの名前を選択する必要があります。

In [ ]:
import boto3
import sagemaker
import os

region = boto3.Session().region_name
sage_client = boto3.Session().client('sagemaker')

#tuning_job_name = 'YOUR-HYPERPARAMETER-TUNING-JOB-NAME'
tuning_job_name = 'sagemaker-tensorflow-181217-1709'

## ハイパーパラメータチューニングジョブの進行状況を追跡する
チューニングジョブを起動すると、`describe_tuning_job API`を呼び出すことでその進行状況を確認できます。 
`describe-tuning-job`からの出力は、チューニングジョブの現在の状態に関する情報を含むJSONオブジェクトです。 
`list_training_jobs_for_tuning_job`を呼び出して、チューニングジョブが起動したトレーニングジョブの詳細なリストを表示できます。

In [ ]:
# run this cell to check current status of hyperparameter tuning job
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)

status = tuning_job_result['HyperParameterTuningJobStatus']
if status != 'Completed':
    print('Reminder: the tuning job has not been completed.')
    
job_count = tuning_job_result['TrainingJobStatusCounters']['Completed']
print("%d training jobs have completed" % job_count)
    
is_minimize = (tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['Type'] != 'Maximize')
objective_name = tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['MetricName']

In [ ]:
from pprint import pprint
if tuning_job_result.get('BestTrainingJob',None):
    print("Best model found so far:")
    pprint(tuning_job_result['BestTrainingJob'])
else:
    print("No training jobs have reported results yet.")

## すべての結果をDataFrameとして取得する
すべてのトレーニング・ジョブのハイパーパラメータと目的のメトリクスをリストアップし、その中でベストなオブジェクトのメトリクスのトレーニング・ジョブを選択することができます。

In [ ]:
import pandas as pd

tuner = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

full_df = tuner.dataframe()

if len(full_df) > 0:
    df = full_df[full_df['FinalObjectiveValue'] > -float('inf')]
    if len(df) > 0:
        df = df.sort_values('FinalObjectiveValue', ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest":min(df['FinalObjectiveValue']),"highest": max(df['FinalObjectiveValue'])})
        pd.set_option('display.max_colwidth', -1)  # Don't truncate TrainingJobName        
    else:
        print("No training jobs have reported valid results yet.")
        
df

## TuningJobの結果と時間を参照
次に、チューニングの作業が進むにつれて対象とするメトリクスがどのように変化するかを示します。 ベイジアン最適化のプロセスでは、より良い結果を得るための一般的な傾向が見られるはずですが、アルゴリズムが既知の良い領域の搾取に対して新しい領域のパラメータ空間のバランスをとる必要があるため、このプロセスは安定しません。 これにより、検索スペースの複雑さに対して、学習ジョブ数が十分であるかどうかの感覚を得ることができます。

In [ ]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

class HoverHelper():

    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append( (k, "@{%s}" % k) )

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools='pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset'):
        return [self.hovertool(), standard_tools]

hover = HoverHelper(tuner)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type='datetime')
p.circle(source=df, x='TrainingStartTime', y='FinalObjectiveValue')
show(p)

## ターゲットメトリクスと個々のハイパーパラメータとの間の相関を分析する
チューニング・ジョブを完了しました。目的のメトリクスとチューニングするように選択した個々のハイパーパラメータとの相関を知りたい場合があります。 この分析では、特定のハイパーパラメータの検索範囲を調整し、別のチューニングジョブを開始することが理にかなっているかどうかを判断することができます。 例として、目的のメトリクスと数値ハイパーパラメータの間に正のトレンドがある場合は、次回のチューニング作業でそのハイパーパラメータのチューニング範囲を高く設定することをお勧めします。

次のセルは、各ハイパーパラメータのグラフを描き、ターゲットメトリクスとの相関を示します。

In [ ]:
ranges = tuner.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get('Values'):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0           
        vals = hp_range['Values']
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args['x_range'] = vals

    # Now plot it
    p = figure(plot_width=500, plot_height=500, 
               title="Objective vs %s" % hp_name,
               tools=hover.tools(),
               x_axis_label=hp_name, y_axis_label=objective_name,
               **categorical_args)
    p.circle(source=df, x=hp_name, y='FinalObjectiveValue')
    figures.append(p)
show(bokeh.layouts.Column(*figures))